BERT는 이미 학습된 모델 가지고 추가 학습(파인튜닝) 하는 것이다.
https://huggingface.co에 가면 목적에 맞는 많은 BERT 모델들이 있다

In [17]:
import urllib.request
import pandas as pd
import numpy as np
from numpy.linalg import norm
from numpy import dot
from sentence_transformers import SentenceTransformer

In [16]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


사전훈련된 BERT를 로드. BERT 항상 사전훈련된 모델에 정확도를 더하기 위해 추가 학습을 한다고 생각하면 된다.
이 모델도 SNIL로 학습 후 STS데이터로 파인튜닝 한 모델

In [4]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\user06\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user06\.cache\huggingface\hub\models--sentence-transformers--xlm-r-100langs-bert-base-nli-stsb-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

다음은 문장단위로 임베딩을 하자. 딥러닝에서 문장의 의미가 하나의 절이 될 수도 있고 두개의 문장이 될 수도 있다.
아래에서는 로우단위를 문장이라고 표현했다.

In [5]:
train_data['embedding'] = train_data.apply(lambda row:model.encode(row.Q), axis=1)

In [6]:
train_data[:5]

,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0,"[0.2017958, -0.034438066, 1.5395721, 0.0106974..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.07716598, -0.034278173, 0.8624423, 0.026360..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.104452476, -0.01243225, 1.0132878, 0.022501..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.09760746, -0.046716824, 0.8936941, 0.021047..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.070028715, 0.031961113, 1.4915429, 4.33996..."


In [7]:
# 코사인 유사도 함수. 이건 그냥 암기. 두문장의 벡터의 유사도를 구하는 것
def cos_sim(a,b):
    return dot(a,b) / (norm(a) * norm(b))

In [8]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x:cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

실제 테스트를 해보자. 테스트는 1.테스트 문장을 임베딩, 2.모델에서 임베딩된 열과 코사인 유사도를 구해서 비교하고 값이 가장 큰 답변을 리턴

In [9]:
return_answer('결혼하고 싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [10]:
return_answer('버티고 버티자. 언젠가 좋은 날이 있겠지')

'좋은 생각만 나면 좋을텐데.'